# Personal Well-being in the UK

*[ONS - Personal well-being in the UK: local authority update, 2015 to 2016](https://www.ons.gov.uk/releases/personalwellbeingintheuklocalauthorityupdate2015to2016)*

Quick notebook to parse out breakdowns of wellbeing metrics by Local Authority.

In [1]:
import pandas as pd

In [5]:
#Download the data
url='https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/wellbeing/datasets/headlineestimatesofpersonalwellbeing/localauthorityupdate2015to2016/headlineestimatesofpersonalwellbeinglocalauthorityupdate2015to2016.xls'
fn=url.split('/')[-1]
!wget {url} -O data/{fn}

In [13]:
#Check the sheetnames
xl=pd.ExcelFile('data/{}'.format(fn))
xl.sheet_names

['Contents',
 'Life Satisfaction - Means ',
 'Life Satisfaction - Thresholds',
 'Worthwhile - Means',
 'Worthwhile - Thresholds',
 'Happy - Mean',
 'Happy - Thresholds',
 'Anxiety - Means',
 'Anxiety - Thresholds']

In [377]:
#Function to load in the data from a particular sheet - some wrangling required to handle the multiindex
#eg see http://stackoverflow.com/questions/39745627/setting-a-row-index-on-and-querying-a-pandas-dataframe-with-multi-index-columns
def getData(typ):
    df=pd.read_excel('data/{}'.format(fn),sheetname='{} - Thresholds'.format(typ),header=None,skiprows=4,na_values=['x'])

    df.dropna(axis=0,inplace=True,how='all')
    df[0][:3]=df[0][:3].fillna(method='backfill',axis=0)
    df[1][:3]=df[1][:3].fillna(method='backfill',axis=0)
    df[:2]=df[:2].fillna(method='ffill',axis=1)
    df=df.T.set_index([0,1]).T
    df=df.rename(columns=lambda x: x.strip())
    df=df.drop(df.index[[0]])

    df.set_index([('Area Codes','Area Codes'),
                  ('Area Names','Area Names')], inplace=True)
    df.index.names = ['Area Codes','Area Names']
    df.sort_index(inplace=True)
    #idx = pd.IndexSlice
    #df.loc[idx['E06000047',:], :]['2011/12*']

    df=df.reset_index()
    df=df[~df['Area Names'].isnull()]
    return df

In [378]:
df.head()

0  Area Codes            Area Names 2011/12*                      2012/13*  \
1                                   Very Low    Low Medium   High Very Low   
7   E06000001            Hartlepool     41.3  21.64  15.36  21.69    46.06   
8   E06000002         Middlesbrough    34.84  19.42  16.77  28.98    42.49   
9   E06000003  Redcar and Cleveland    39.35  20.34  18.42  21.89    41.24   
10  E06000004      Stockton-on-Tees    37.59  21.98  20.46  19.97    41.55   
11  E06000005            Darlington    42.72  20.13  16.58  20.57    41.97   

0                        ...   2013/14*        2014/15*                       \
1     Low Medium   High  ...     Medium   High Very Low    Low Medium   High   
7   18.29  14.69  20.96  ...      15.06  21.93    48.52  17.53   13.6  20.35   
8   19.55  14.61  23.35  ...       18.2  23.69    42.74  18.35  16.87  22.03   
9   18.86  18.03  21.87  ...      17.54  20.44    47.39  19.59  14.64  18.38   
10  19.21  18.94  20.29  ...      17.07  20.31    42.52  21.11  16.16   20.2   
11  21.76  17.14  19.13  ...      14.89  20.98    40.48   22.8  16.26  20.46   

0  2015/16*                       
1  Very Low    Low Medium   High  
7     44.78  17.43  16.73  21.05  
8     43.43  19.85  16.33   20.4  
9     44.01  17.54  18.21  20.24  
10    45.64  16.09  17.42  20.85  
11    43.95  20.05  15.35  20.66  

[5 rows x 22 columns]

In [379]:
#PUT YOUR SEARCH QUERY TERMS HERE
area='Isle of Wight'
period='2014/15*'

In [380]:
from collections import OrderedDict

clists={'Anxiety':['Very Low','Low','Medium','High'],
        'Life Satisfaction':['Low','Medium','High','Very High'],
        'Happy':['Low','Medium','High','Very High'],
        'Worthwhile':['Low','Medium','High','Very High']
        }

for t in ['Life Satisfaction','Worthwhile','Happy','Anxiety']:
    txtlist=[]
    df=getData(t)
    pc=OrderedDict((b,df[df['Area Names']==area][period][b].iloc[0]) for b in clists[t])
    txtlist.append('''
When it comes to {typ}, in the {period}, the percentage breakdowns for {area} residents were as follows: {pc}
'''.format(typ=t,period=period,area=area,pc=', '.join(['{}: {}%'.format(p,pc[p]) for p in pc])) )
    print('\n'.join(txtlist))


When it comes to Life Satisfaction, in the 2014/15*, the percentage breakdowns for Isle of Wight residents were as follows: Low: 5.41%, Medium: 13.46%, High: 50.73%, Very High: 30.4%


When it comes to Worthwhile, in the 2014/15*, the percentage breakdowns for Isle of Wight residents were as follows: Low: 3.72%, Medium: 11.05%, High: 44.43%, Very High: 40.8%


When it comes to Happy, in the 2014/15*, the percentage breakdowns for Isle of Wight residents were as follows: Low: 8.15%, Medium: 15.64%, High: 37.19%, Very High: 39.03%


When it comes to Anxiety, in the 2014/15*, the percentage breakdowns for Isle of Wight residents were as follows: Very Low: 45.62%, Low: 20.67%, Medium: 15.59%, High: 18.12%



In [384]:
#who's stressed?
df=getData('Anxiety')
df[['Area Codes','Area Names','2015/16*']].sort_values([('2015/16*', 'High')], ascending=False).head()

0   Area Codes               Area Names 2015/16*                     
1                                       Very Low    Low Medium   High
312  E09000013  Hammersmith and Fulham     27.86  22.58  19.01  30.56
144  E07000109                Gravesham    40.61    NaN    NaN  30.15
187  E07000152    East Northamptonshire    32.11  24.93    NaN   28.9
49   E06000043        Brighton and Hove    30.09  23.25  18.14  28.51
310  E09000011                Greenwich    32.26  22.65   16.6  28.49

In [385]:
#who's chilled?
df[['Area Codes','Area Names','2015/16*']].sort_values([('2015/16*', 'High')], ascending=True).head()

0   Area Codes             Area Names 2015/16*                     
1                                     Very Low    Low Medium   High
304  E09000005                 Brent     37.93  30.56  19.65  11.86
54   E06000049          Cheshire East    44.08   27.3  16.55  12.07
123  E07000084  Basingstoke and Deane    47.98  22.38   17.3  12.33
104  E07000065                Wealden    46.78   22.9   17.9  12.42
72   E07000026              Allerdale    42.98  23.76  20.59  12.66